In [1]:
from piron import FitsArray, Combine, Calibration, Calculator, Coordinates

In [2]:
fa = FitsArray.from_pattern("data/*.fit")
fa

FitsArray(@: 139644753501472, nof: 574)

In [3]:
groupped = fa.groupby("IMAGETYP")
groupped

{'Bias Frame': FitsArray(@: 139644307401280, nof: 10),
 'Dark Frame': FitsArray(@: 139644307397296, nof: 30),
 'Flat Field': FitsArray(@: 139644305448128, nof: 24),
 'Light Frame': FitsArray(@: 139644305564016, nof: 510)}

In [4]:
zero_c = Combine(groupped['Bias Frame'])
master_zero = zero_c.zerocombine("median", output="data/out/master_zero.fit")
master_zero

Fits(@: 139644307495120, file: [OBS_PATH]/data/out/master_zero.fit)

In [5]:
darks = groupped['Dark Frame'].groupby("EXPTIME")
darks

{15.0: FitsArray(@: 139644306239024, nof: 10),
 20.0: FitsArray(@: 139644306204096, nof: 10),
 30.0: FitsArray(@: 139644306203808, nof: 10)}

In [6]:
flats = groupped['Flat Field'].groupby("FILTER")
flats

{'Bessel_B': FitsArray(@: 139644306231440, nof: 8),
 'Bessel_U': FitsArray(@: 139644306179648, nof: 8),
 'Bessel_V': FitsArray(@: 139644306186320, nof: 8)}

In [7]:
data = groupped['Light Frame'].groupby(["EXPTIME", "FILTER"])
data

{(15.0, 'Bessel_V'): FitsArray(@: 139644306180752, nof: 170),
 (20.0, 'Bessel_B'): FitsArray(@: 139644306198912, nof: 170),
 (30.0, 'Bessel_U'): FitsArray(@: 139644306173584, nof: 170)}

In [11]:
new_fits = []
for (exp, fltr), images in data.items():
    dark_c = Combine(darks[exp])
    master_dark = dark_c.darkcombine("median", output=f"data/out/master_dark_{exp}.fit", override=True)
    
    flat_c = Combine(flats[fltr])
    master_flat = flat_c.flatcombine("median", output=f"data/out/master_flat_{fltr}.fit", override=True)
    
    calib = Calibration(images)
    calibrated = calib.calibrate(zero=master_zero, dark=master_dark, flat=master_flat, output=f"data/cali/")

In [13]:
calibrated.header

,SIMPLE,BITPIX,NAXIS,NAXIS1,NAXIS2,EXTEND,ORIGIN,DATE,IRAF-TLM,OBJECT,...,WAT0_001,WAT1_001,WAT2_001,ZEROCOR,DARKCOR,FLATCOR,CCDSEC,CCDMEAN,CCDMEANT,CCDPROC
image,,,,,,,,,,,,,,,,,,,,,
[OBS_PATH]/data/cali/V523_Cas-001U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:01:45,2022-12-06T06:01:45,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:01 Zero level correction image is /m...,Dec 6 9:01 Dark count correction image is /m...,Dec 6 9:01 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",57.36434,1354784505,Dec 6 9:01 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-002U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:01:46,2022-12-06T06:01:46,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:01 Zero level correction image is /m...,Dec 6 9:01 Dark count correction image is /m...,Dec 6 9:01 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",55.54247,1354784506,Dec 6 9:01 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-003U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:01:46,2022-12-06T06:01:46,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:01 Zero level correction image is /m...,Dec 6 9:01 Dark count correction image is /m...,Dec 6 9:01 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.85281,1354784506,Dec 6 9:01 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-004U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:01:46,2022-12-06T06:01:46,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:01 Zero level correction image is /m...,Dec 6 9:01 Dark count correction image is /m...,Dec 6 9:01 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.66858,1354784506,Dec 6 9:01 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-005U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:01:47,2022-12-06T06:01:47,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:01 Zero level correction image is /m...,Dec 6 9:01 Dark count correction image is /m...,Dec 6 9:01 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.87293,1354784507,Dec 6 9:01 CCD processing done
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[OBS_PATH]/data/cali/V523_Cas-166U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:02:38,2022-12-06T06:02:38,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:02 Zero level correction image is /m...,Dec 6 9:02 Dark count correction image is /m...,Dec 6 9:02 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",1215.21100,1354784558,Dec 6 9:02 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-167U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:02:38,2022-12-06T06:02:38,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:02 Zero level correction image is /m...,Dec 6 9:02 Dark count correction image is /m...,Dec 6 9:02 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",1586.88500,1354784558,Dec 6 9:02 CCD processing done
[OBS_PATH]/data/cali/V523_Cas-168U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-06T06:02:39,2022-12-06T06:02:38,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 6 9:02 Zero level correction image is /m...,Dec 6 9:02 Dark count correction image is /m...,Dec 6 9:02 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",2081.61500,1354784559,Dec 6 9:02 CCD processing done
